# Entropy estimation

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import math

In [ ]:
lag = 16
stats_count = lag + 4
output_dim = 1
input_data_dim = 20
training_set_ratio = 0.8
#batch_size

## Data-Preprocessing

In [ ]:
import math

training_set_ratio = 0.8

with open('random_files/dev-random.bin','rb') as fr:
    fr.seek(0,2)
    training_set_size = math.floor(0.8*fr.tell())
    
    
    fr.seek(0,0)
    data = fr.read(training_set_size)
    

In [ ]:
byte_string = "{0:08b}".format(int.from_bytes(data,'big'))

In [ ]:
X_D = [float(char) for char in byte_string[:-((training_set_size*8)%(input_data_dim+1))]]

In [ ]:
Y = X_D[20::21]

In [ ]:
del X_D[20::21]

In [ ]:
X_D = torch.tensor(X_D)

In [ ]:
X_D = X_D.view(((training_set_size*8)//(input_data_dim+1)), input_data_dim)

In [ ]:
X_D

In [ ]:
Y = torch.tensor(Y)

In [ ]:
Y = Y.view(((training_set_size*8)//(input_data_dim+1)), 1)

In [ ]:
Y

## Neural Network

In [ ]:
class Neural_Network(nn.Module):
    def __init__(self, stats_count, input_data_dim, num_classes):
        super(Neural_Network, self).__init__()
        self.fc1 = nn.Linear(stats_count, 20)
        self.fc2 = nn.Linear(20, 1)
        self.fc3 = nn.Linear(input_data_dim+1, 128)
        self.fc4 = nn.Linear(128, 20)
        self.fc5 = nn.Linear(20, num_classes)
    
    
    def forward(self, stats, input_data):
        x = F.relu(self.fc1(stats))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(torch.cat((input_data, x), dim=1)))
        x = F.relu(self.fc4(x))
        x = F.leaky_relu(self.fc5(x))        #log_softmax
        return x

model = Neural_Network(stats_count, input_data_dim, output_dim)
print(model)

In [ ]:
loss_function = nn.L1Loss()
total_epochs = 20
learning_rate = 0.001    #update the equation
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
X_S = torch.randn(((training_set_size*8)//(input_data_dim+1)), 20)

In [ ]:
X_S

In [ ]:
output = model(X_S,X_D)

In [ ]:
for epoch in range(total_epochs):
    model.zero_grad()
    output = model(X_S, X_D)
    
    loss = loss_function(output, Y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(loss)